<a href="https://colab.research.google.com/github/lseidy/Aulas-AED/blob/master/zoologico_SO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Autores: Lucas Seidy e Willian Rodrigues

import time
import random
import threading
from functools import partial
import sys
import datetime

flag = True # variavel que controla o termino da execução do zoologico
Dias = 3    #numero de dias na execução
#definindo classe Estoque
class Estoque:
    def __init__(self, estoque: int):
        self.estoque = list([estoque]*3)
        self.pedido = threading.Event()

    def control_estoque(self, id: int):     # retorna o valor do estoque acessado
        return self.estoque[id]
    def requere_reposicao(self):            # aciona o reabastecimento do fornecedor
        self.pedido.set()
    def get_pedido(self):                   # retorna a variavel de sincronização de threads
        return self.pedido
    def set_estoque(self, id: int):         # retirna 1 do estoque acessado
        self.estoque[id] -= 1


#definindo classe Fornecedor
class FornecedorThread(threading.Thread):
    def __init__(self, estoque: Estoque):
        threading.Thread.__init__(self)
        self.estoque_zoo = estoque

    def set_reposicao(self):    # repoe todos os estoques
        if flag == True:
            self.estoque_zoo.estoque = [30 for i in range(3)]
            print("\n------------------------- [Realizou Reposicao]----------------------------\n")
            self.estoque_zoo.pedido.clear() # volta a indicar a espera para execução da thread
        else:
            return

    def run(self): # função de start da thread
        while(flag):
            self.estoque_zoo.pedido.wait()  # aguarda a sinalização pra começar o processo
            self.set_reposicao()
class Zoo:
    def __init__(self, comedouro: int, estoque: Estoque):
        self.comedouro = list([10]*comedouro)                           #Buffer utilizado cada indice um buffer por animal
        self.estoque = estoque
        self.inicio = datetime.datetime.now().timestamp()
        self.sem_vazio = list([threading.Semaphore(0)]*comedouro)      #define lista de semaforos vazios (1 por comedouro)
        self.sem_cheio = list([threading.Semaphore(10)]*comedouro)     #define lista de semaforos cheios (1 por comedouro)
        self.mutex_lock = list([threading.Lock()]*comedouro)           #define lista mutex do veterinario (1 por comedouro)
        self.mutex_lock_vet = list([threading.Lock()]*comedouro)       #define lista de semaforos cheios (1 por comedouro)

    def get_comedouro(self, id: int):       # retorna conteudo do comedouro acessado
        return self.comedouro[id]
    def get_lock_vet(self, id: int):        # retorna o mutex indicado
        return self.mutex_lock_vet[id]
    def get_sem_cheio(self, id: int):       # retorna o semaforo cheio indicado
        return self.sem_cheio[id]
    def get_sem_vazio(self, id: int):       # retorna o semaforo vazio indicado
        return self.sem_vazio[id]
    def get_lock(self, id: int):            # retorna mutex indicado
        return self.mutex_lock[id]

#definindo classe Animal
class AnimalThread(threading.Thread):
    def __init__(self, zoo: Zoo, id: int,especie: str):
        threading.Thread.__init__(self)
        self.zoo = zoo
        self.especie = especie
        self.alimentacao = 0
        self.id = id
        self.exibicao = 0
        self.dormido = 0

    def set_alimentacao(self, qtd):     # aumenta a qtd de alimentos consumidos pelo animal
        self.alimentacao += qtd

    def get_alimentacao(self):          # retorna o numero de alimentos consumidos pelo animal
        return self.alimentacao

    def exibir(self):                   # realiza função basica do animal
        print("-------------------")
        print(f"\n\n[ {self.especie} esta se exibindo ]\n\n")
        print("-------------------")
        self.exibicao += 1
        time.sleep(random.randint(1,2))
    def comer(self, id: int):           # realiza função basica do animal
        if id == 0:
            qtd = random.randint(2,5)
        elif id == 1:
            qtd = random.randint(1,2)
        else:
            qtd = random.randint(2,4)

        # reliza produtor-consumidor
        self.zoo.get_sem_cheio(self.id).acquire()
        self.zoo.get_lock(self.id).acquire()

        if(self.zoo.get_comedouro(id) >= qtd):
            print("-------------------")
            print(f"\nComedouro com [{self.zoo.get_comedouro(id)}] und. de comida")
            self.zoo.comedouro[id] -= qtd
            print(f"Restou [{self.zoo.get_comedouro(id)}] und. de comida apos {self.especie} comer")
            self.set_alimentacao(qtd)
            print(f"\n[{self.especie} comeu {self.get_alimentacao()} alimentos ate o momento]\n")
            print("-------------------")
        else:
            print(f"\n      [Nao ha comida no comedouro {id} para o {self.especie}]\n")
        self.zoo.get_lock(self.id).release()
        self.zoo.get_sem_vazio(self.id).release()
        # desbloqueia semaforos
    def dormir(self, id: int):              # realiza função basica do animal
        if id ==0:
            sleep = random.randint(8,12)
        elif id == 1:
            sleep = random.randint(6,10)
        else:
            sleep = random.randint(4,8)
        self.dormido += sleep
        print(f"\n{self.especie} dormindo\n")
        time.sleep(sleep)
        print(f"\n{self.especie} acordando, dormiu {sleep} horas\n")

    def run(self):                  # função de start da thread animal
        global flag
        print(f"\n[             Animal {self.especie} criado            ]\n")
        i = 1
        while(flag):                # realiza enquanto a flag for true
            if i <= Dias * 2:
                if(i%2 == 1):
                    print(f"************************************** [ EXECUTANDO DIA {int(i/2+1)} ] *************************************")
                self.comer(self.id)
                self.exibir()
                self.comer(self.id)
                self.dormir(self.id)
                if i % 2 == 0:
                    print(f"\n-------------------------- [{self.especie} fez a rotina {i/2} vezes] --------------------------\n")
                i += 1
            else:
                print(f"\n----------------------- [Animal {self.especie} ja fez a rotina dos dias] -------------------------\n")
                with open('animais.txt', 'a') as f:
                    f.write("\n---------------------\n")
                    f.write(f"[Animal: {self.especie} comeu {self.get_alimentacao()} alimentos]\n")
                    f.write(f"[Exibicao: {self.especie} se exibiu {self.exibicao} vezes]\n")
                    f.write(f"[Dormir: {self.especie} dormiu {self.dormido} horas]\n")
                    f.write("---------------------\n")
                return
#define classe Veterinario
class VeterinarioThread(threading.Thread):
    def __init__(self, zoo: Zoo,num: int):
        threading.Thread.__init__(self)
        self.vet = zoo
        self.id = num
        self.execucao = 0
        self.comida =0
    def reabastece_comedouro(self, id: int):                # reabastece o comedouro
        if self.vet.estoque.control_estoque(id) > 0:
            print("-------------------")
            print(f"Vet {self.id} executando\nComida no Comedouro[{id}] antes : {self.vet.get_comedouro(id)}")
            self.vet.estoque.set_estoque(id)
            self.vet.comedouro[id] += 1
            self.comida += 1
            self.execucao += 1
            print(f"Veterinario colocou 1und no comedouro [{id}]")
            print(f"Comida no Comedouro[{id}] depois : {self.vet.get_comedouro(id)}")
            print(f"Estoque [{id}] Depois: ", self.vet.estoque.control_estoque(id), "\n")
            print("-------------------")
        else:
            print("\n--------------------------- [Estoque Vazio Solicitando Reposicao] ------------------------------\n")
            self.vet.estoque.requere_reposicao()

    def verifica_comedouro(self, id: int):      # verifica se pode reabastecer o comedouro
        # realiza produtor-consumidor
        self.vet.get_sem_vazio(id).acquire()
        self.vet.get_lock_vet(id).acquire()

        if (self.vet.get_comedouro(id) < 10):
            self.reabastece_comedouro(id)

        self.vet.get_lock_vet(id).release()
        self.vet.get_sem_cheio(id).release()
        # desbloqueia semaforos

    def run(self):          #função de start do veterinario
        global flag
        while(flag):
            for i in range(3):
                self.verifica_comedouro(i)

        with open('veterinarios.txt', 'a') as f:
            f.write("\n---------------------\n")
            f.write(f"[Veterinario: {self.id} executou {self.execucao} vezes]\n")
            f.write(f"[Reabasteceu Comedouros: {self.comida}  vezes]\n")
            f.write("---------------------\n")

estoque = Estoque(30)   # cria objeto estoque
zoo = Zoo(3, estoque)   # cria objeto zoologico

vet = []
leao = []
suricato = []
avestruz = []
for i in range(4):      # define lista de threads de leoes
    leao.append(AnimalThread(zoo, 0,"Leao "+ str(i)))

for i in range(10):     # define lista de threads de suricatos
    suricato.append(AnimalThread(zoo, 1, "Suricato "+ str(i)))
for i in range(7):      # define lista de threads de avestruzes
    avestruz.append(AnimalThread(zoo, 2, "Avestruz "+ str(i)))

fornecedor = FornecedorThread(estoque)  #define thread de fornecedor
vet.append(VeterinarioThread(zoo, 0))   #define lista de threasd de veterinario
vet.append(VeterinarioThread(zoo, 1))

for i in vet+leao+suricato+avestruz:    #inicializa threads
    i.start()
fornecedor.start()

time.sleep(Dias*24)      # aguarda X tempo para terminar
flag = False        # infica termino da execução

for i in vet+leao+suricato+avestruz:    # aguarda termino das threads
    zoo.get_sem_cheio(0).release()
    zoo.get_sem_vazio(0).release()
    zoo.get_sem_cheio(1).release()
    zoo.get_sem_vazio(1).release()
    zoo.get_sem_cheio(2).release()
    zoo.get_sem_vazio(2).release()
    i.join()
    print(f"{i} acabou")
fornecedor.estoque_zoo.requere_reposicao()
fornecedor.join()

Streaming output truncated to the last 5000 lines.
-------------------


[ Avestruz 1 esta se exibindo ]


-------------------
************************************** [ EXECUTANDO DIA 1 ] *************************************
-------------------************************************** [ EXECUTANDO DIA 1 ] *************************************

Vet 1 executando
Comida no Comedouro[2] antes : 5
Veterinario colocou 1und no comedouro [2]
Comida no Comedouro[2] depois : 6
Estoque [2] Depois:  28 

-------------------
-------------------

Comedouro com [6] und. de comida
Restou [4] und. de comida apos Avestruz 2 comer

[Avestruz 2 comeu 2 alimentos ate o momento]

-------------------
-------------------


[ Avestruz 2 esta se exibindo ]


-------------------
-------------------

Comedouro com [4] und. de comida
Restou [2] und. de comida apos Avestruz 3 comer

[Avestruz 3 comeu 2 alimentos ate o momento]

-------------------
-------------------


[ Avestruz 3 esta se exibindo ]


---------------